# Simrunner - Example use with Tuflow
Author: Tom Norman  
Version 0.1.0 (Prerelease)

Simrunner is a framework that can work with different simulation executables. Tuflow was my use case, the tuflowrunner module is built upon simrunner to execute tuflow models in python. An example of what can be achieved with tuflowrunner in particular and Simrunner in general is given in this notebook. When getting started and familiar with the simrunner package, this notebook can be used as a template.

__Disclaimer__: 
This package, or any of the work associated with simrunner has absolutely no affiliation with Tuflow or BMT. 

In [2]:
import simrunner.tuflow as tfr

## Parameters

The executable parameters of the simulation are set in the Parameters object. This is then given to the Runner. 

Set the Parameters of the run. Paths need to be entered as string literals to avoid any errors in Windows (put an 'r' in front). 

In [3]:
parameters: tfr.Parameters = tfr.Parameters(
    exec_path = r"C:/Program Files/TUFLOW",      # (Required) The path to the TUFLOW executable.
    root = r"../tests/data/tuflow/group_tcf",    # (Required) The root of the model, location of the .tcf file.
    version = r"2020-10-AD",                     # (Required) The version of the engine.
    engine = r"DP",                              # (Reguired) SP or DP.
    flags = ['-t', '-b'],                        # (Optional [None]) Any number of flags as a list.
    async_runs = 2,                              # (Optional [1]) How many models to run at once.
    stdout = r'../trash/stdout',                 # (Optional [root]) Where to put the model output (what TUFLOW prints to the console).
    group = r"group_tag",                        # (Optional [None]) The name of the tcf file grouping identifier.
    run_args = ['s1', 'e1']                      # (Optional [Auto]) The run arguments the runner is expecting.
)

__exec_path__ (Required)  
The path to the Tuflow executables. The location where all the different versions of the executables live.   

__root__ (Required)  
The root of the project. The location of the .tcf.  

__version__ (Required)  
The version number of the Tuflow executable. Assumes that the executable resides with the version folder. This is standard when downloading the executable from Tuflow. 

__engine__ (Required)  
Either single percision (SP) or double percision (DP) engine.

__flags__ (Optional)  
The calling flags of Tuflow, these are the -b, -t, ... flags that are passed to Tuflow in the CLI.

__asyc_runs__ (Optional)  
The number of models that you wish to run at once. 

__stdout__ (Optional)  
This library does not populate a console when Tuflow runs. Instead it uses interactive widgets and puts the output of the console in a text file. This is the file path to this output location. The default path is in the root directory. 

__group__ (Optional)  
It may be desirable in some situations to group control files in the same root directory. Typically the TCFs within the project root are considered part of the same group and the TCFs share the same set of run arguments. When multiple TCFs in the root directory are called in different ways, a group should be given by prefixing the group id to the TCF. For example the TCF with names:
1. __"GY\_~s1~\_~e1~\_~e2~\_01.tcf"__ 
2. __"GY\_~s1~\_~e1~\_~e2~\_02.tcf"__ 
3. __"GX\_~s1~\_~s2~\_~e1~\_~e2~\_01.tcf\"__.   

The __"GY"__ and __"GX"__ are the group identifiers. When this parameter is omitted as single group is assumed.   

Groups must all have the same set of run arguments, so if different TCFs have different run arguments then they should be in different groups.

__run_args__ (Optional)  
Define the TCF expected run arguments ahead of time. If the folder structure is setup correctly and each group has the same set of run arguments than Simrunner will determine the run arguments automatically. The use of this parameter should be avoided and is included only for edge use cases or legacy projects needed to be ported.  

## Run  
A run is a single run of the model, with a unique set of parameters that modify the model's behaviour. Multiple runs can be staged to a runner.  

With the tuflow example these are the parameters you pass to the command line when running Tuflow. Just like when using the command line, the run arguments need to match the tfc otherwise an error will occur when trying to stage the runs.  

The parameters are given to the Run as __key=value__ pairs. The parameter place holder is the key, the value is the parameter's value. In the below example s1="DES" is functionally equivalent to '-s1 DES'. 

In [3]:
run = tfr.Run(s1="DES", e1="M720", e2="2090")

## Runner
The Runner is the main object wich organises your runs and responsible for executing all your staged runs.  

Create a new runner with the parameters defined above. 

In [4]:
runner = tfr.Runner(parameters)

Stage the above run in the runner.

In [5]:
runner.stage(run)

Just like with Batch Scripts you can loop through a bunch of scenarios and stage them in the runner. 

In [6]:
scenarios = ['EX', 'DES']
for s in scenarios:
    run = tfr.Run(
        s1=s,
        e1='M60',
        e2='CC'
    )
    runner.stage(run)

Or if your runs are already in a list. A list of runs can be staged all at once.

In [7]:
runner_subset = tfr.Runner(parameters)
des_runs = runner.get_runs('DES')
runner_subset.stage(des_runs) 

runner_subset.get_runs()

[{'s1': 'DES', 'e1': 'M720', 'e2': '2090'},
 {'s1': 'DES', 'e1': 'M60', 'e2': 'CC'}]

To make it easy to get runs, you can get a run from a runner by index. Get the first run added to the runner.

In [8]:
run_indexed = runner[0]
run_indexed

{'s1': 'DES', 'e1': 'M720', 'e2': '2090'}

Or use slicing. Get the first two runs added to the runner.

In [9]:
runner_sliced = runner[:2]
runner_sliced

[{'s1': 'DES', 'e1': 'M720', 'e2': '2090'},
 {'s1': 'EX', 'e1': 'M60', 'e2': 'CC'}]

To see what runs have been added to the runner.

In [10]:
runner.get_runs()

[{'s1': 'DES', 'e1': 'M720', 'e2': '2090'},
 {'s1': 'EX', 'e1': 'M60', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M60', 'e2': 'CC'}]

Run the staged runs, with no associated run number.

In [ ]:
runner.run()

To run a specific run number.

In [ ]:
runner.run('01')

To run multiple run numbers, this will alternate run numbers as the runner proceeds through the runs. 

In [ ]:
runner.run('01', '02')

And to run the run numbers in sequence. 

In [ ]:
runner.run('01')
runner.run('02')

You can create a new tuflow runner from other runners. The staged runs are inherited from runners it was created with runners. 

In [13]:
runner_1 = tfr.Runner(parameters)
runner_2 = tfr.Runner(parameters)
runner_3 = tfr.Runner(parameters)

runner_1.stage(tfr.Run(s1='EX', e1='M120', e2='CC'))
runner_2.stage(tfr.Run(s1='DES', e1='M120', e2='CC'))
runner_3.stage(tfr.Run(s1='DES', e1='M45', e2='NC'))

runner_all = tfr.Runner(parameters, runner_1, runner_2, runner_3)

runner_all.get_runs()

[{'s1': 'EX', 'e1': 'M120', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M120', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M45', 'e2': 'NC'}]

There is no point running the exact same model twice, so duplicates are not added. 'runner_all' already has these three models so they wont be added again.

In [14]:
runner_all.stage(tfr.Run(s1='EX', e1='M120', e2='CC'))
runner_all.stage(tfr.Run(s1='DES', e1='M120', e2='CC'))
runner_all.stage(tfr.Run(s1='DES', e1='M45', e2='NC'))

runner_all.get_runs()

[{'s1': 'EX', 'e1': 'M120', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M120', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M45', 'e2': 'NC'}]

Filter runs based on any model parameters. Lets get all the DES runs.

In [15]:
des_runs = runner.get_runs('DES')
des_runs

[{'s1': 'DES', 'e1': 'M720', 'e2': '2090'},
 {'s1': 'DES', 'e1': 'M60', 'e2': 'CC'}]

Now get all runs that have the parameter 'DES' or 'EX'.

In [16]:
des_ex_runs = runner.get_runs('DES', 'EX')
des_ex_runs

[{'s1': 'DES', 'e1': 'M720', 'e2': '2090'},
 {'s1': 'EX', 'e1': 'M60', 'e2': 'CC'},
 {'s1': 'DES', 'e1': 'M60', 'e2': 'CC'}]

Setting 'any=False' filters runs that must match ALL listed arguments. No runs satisfy both DES and EX scenarios.

In [17]:
no_runs = runner.get_runs('DES', "EX", any=False)
no_runs

[]

Remove runs from a runner. Lets remove the DES runs from the runner, we retrieved above.

In [18]:
runner_remove = tfr.Runner(parameters, runner)
runner_remove.remove_runs(des_runs)
runner_remove.get_runs()

[{'s1': 'EX', 'e1': 'M60', 'e2': 'CC'}]

Make testing straight forward, by setting parameter on the fly.

In [ ]:
TEST = True

if TEST:
    parameters.flags = ['-t']
    runner.run()
else:
    parameters.flags = ['-b']
    runner.run()

Change versions on the fly.

In [ ]:
parameters.version = '2020-10-AF'
runner.run()